## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-03-01-05-11-35 +0000,wapo,"Iranian President Masoud Pezeshkian, judiciary...",https://www.washingtonpost.com
1,2026-03-01-05-11-35 +0000,wapo,Live updates: Iran’s supreme leader killed in ...,https://www.washingtonpost.com/world/2026/02/2...
2,2026-03-01-05-06-34 +0000,nyt,Live Updates: Iran Says Supreme Leader Killed ...,https://www.nytimes.com/live/2026/02/28/world/...
3,2026-03-01-05-05-19 +0000,nyt,The C.I.A. Helped Pinpoint a Gathering of Iran...,https://www.nytimes.com/2026/03/01/us/politics...
4,2026-03-01-05-01-19 +0000,nyt,"40 Years and 10,000 Broadcasts In, They’re Sti...",https://www.nytimes.com/2026/03/01/business/me...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2613/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,iran,89
42,trump,41
22,strikes,35
20,attack,20
0,iranian,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
48,2026-03-01-01-01-13 +0000,wapo,"See where U.S., Israeli strikes have hit Iran ...",https://www.washingtonpost.com/investigations/...,248
1,2026-03-01-05-11-35 +0000,wapo,Live updates: Iran’s supreme leader killed in ...,https://www.washingtonpost.com/world/2026/02/2...,234
38,2026-03-01-01-52-12 +0000,nypost,Multiple members of Supreme Leader Ayatollah A...,https://nypost.com/2026/02/28/world-news/multi...,216
199,2026-02-28-10-14-09 +0000,nypost,"Read, watch President Trump’s full speech afte...",https://nypost.com/2026/02/28/us-news/read-pre...,211
216,2026-02-28-07-02-26 +0000,latimes,Iran's supreme leader killed in U.S. and Israe...,https://www.latimes.com/world-nation/story/202...,207


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
48,248,2026-03-01-01-01-13 +0000,wapo,"See where U.S., Israeli strikes have hit Iran ...",https://www.washingtonpost.com/investigations/...
216,118,2026-02-28-07-02-26 +0000,latimes,Iran's supreme leader killed in U.S. and Israe...,https://www.latimes.com/world-nation/story/202...
32,65,2026-03-01-02-10-50 +0000,nyt,See a Breakdown of Ayatollah Khamenei and Othe...,https://www.nytimes.com/2026/02/28/world/middl...
23,41,2026-03-01-02-53-00 +0000,wsj,Democrats in Congress led the charge to releas...,https://www.wsj.com/politics/policy/epstein-fi...
159,41,2026-02-28-15-04-01 +0000,nypost,Saudi Arabia condemns Iran’s attacks on Gulf n...,https://nypost.com/2026/02/28/us-news/saudi-ar...
81,31,2026-02-28-23-00-13 +0000,nyt,"Trump, the Self-Declared Peace President, Goes...",https://www.nytimes.com/2026/02/28/us/politics...
130,30,2026-02-28-18-40-55 +0000,nypost,Nancy Mace bashes Ilhan Omar over Iran attack ...,https://nypost.com/2026/02/28/us-news/nancy-ma...
49,29,2026-03-01-01-00-00 +0000,wsj,Inheritance is one of the last reliable ways f...,https://www.wsj.com/economy/housing/in-califor...
123,29,2026-02-28-19-13-49 +0000,latimes,"He saw an abandoned trailer. Then, he uncovere...",https://www.latimes.com/california/story/2026-...
51,25,2026-03-01-00-51-51 +0000,nypost,US Ambassador tells off Iranian diplomat in fi...,https://nypost.com/2026/02/28/world-news/us-am...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
